# **Assignment \#4**: Machine Learning MC886/MO444
University of Campinas (UNICAMP), Institute of Computing (IC)

Prof. Marcelo Reis, 2022s1



In [ ]:
# TODO: RA & Name
print('RA1: ' + 'Nome1')
print('RA2: ' + 'Nome2')

## Objective

Explore **clustering techniques** (https://scikit-learn.org/stable/modules/clustering.html) and come up with the best possible model to the problem, avoiding overfitting. In particular, build an clustering model to analisys brazilian music styles [Brazilian Lyrics dataset](https://arxiv.org/pdf/2003.05377.pdf).

# Dataset

Brazilian Lyrics is a dataset build by Romulo Costa and Raul Araujo (PUC-RJ), consisting of 138,368 songs distributed across 14 genres. The authors split the dataset into tree partitions: training (96,857 samples), validation (27,673 samples), and test (13,838 samples).

**Dataset Information:**

- You should respect the following traininig/validation/test split. Avoid overfitting.
- The data is available at: https://drive.google.com/drive/folders/1b681ChByK737CpASYImFB4GfuqPdrvBN
- You should evaluate the models using the Silhouette Coefficient.

## Deadline

Wednesday, June 29th 11:59pm.

Penalty policy for late submission: You are not encouraged to submit your assignment after due date. However, in case you did, your grade will be penalized as follows:
- June 30th 11:59pm : grade * 0.25


## Submission

On Google Classroom, submit your Jupyter Notebook (in Portuguese or English).

**This activity is NOT individual, it must be done in pairs (two-person group).**

## Activities


In [ ]:
!pip install transformers[sentencepiece] --quiet

     |████████████████████████████████| 4.4 MB 8.8 MB/s 
     |████████████████████████████████| 596 kB 69.9 MB/s 
     |████████████████████████████████| 86 kB 5.4 MB/s 
     |████████████████████████████████| 6.6 MB 45.0 MB/s 
     |████████████████████████████████| 1.2 MB 62.7 MB/s 


### 1. Text Embedding

To generate the embeddings of the songs we used the model (Bertimbau)[https://github.com/neuralmind-ai/portuguese-bert] a model based on BERT for Portuguese.
We embedding the title only, the music only and a combination of title + music.
The embedding vector used is the one referring to the special token [CLS] that generates a compact representation of the input information.

Use one of the available techniques to generate the input text embedding:

* Title embedding
* Music embedding
* Title embedding + music

Analyze the distributions by plotting the embeddings generated with the aid of a dimensionality reduction algorithm.


In [ ]:
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
import multiprocessing as mp

import torch
from transformers import BertTokenizerFast, BertModel

from torch.utils.data import DataLoader
from torch.utils.data import Dataset

model_ckpt = 'neuralmind/bert-base-portuguese-cased'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#### Code used to generate embeddings

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Dataset/vagalume.dev.csv')
print("Length dataset: ", df.shape)

tamanho treino:  (13838, 4)


In [ ]:
df[df['music_lyric'].isnull()]

,artist_name,genre,music_title,music_lyric
149,Ana Carolina e Seu Jorge,mpb,Comparsas / O Pequenez E O Pit Bull,NaN
355,Ana Carolina,pop,10 Minutos,NaN
399,Ana Carolina,pop,"Tá Rindo, É?",NaN
747,Ana Carolina,pop,Dentro,NaN
777,Ana Carolina,pop,Vê Se Me Esquece,NaN
1711,Ana Carolina,pop,Torpedo,NaN
3926,Ana Carolina,pop,Só Fala Em Mim,NaN
5522,Roberto Carlos,jovem-guarda,O Careta,NaN
5765,Ana Carolina,pop,É Hora da Virada,NaN
7981,Ana Carolina,pop,Elevador,NaN


In [ ]:
df = df.dropna()

In [ ]:
class MusicaDataset(Dataset):

    def __init__(self, data):
        """
        You can try some variations of the text that will be
        used. Ex: Title + Music, Music, Title
        """
        #self.texts = data['music_title'].values
        #self.texts = data['music_lyric'].values
        self.texts = (data['music_title'] + " " + data['music_lyric']).values

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):

        text = self.texts[idx]

        return text

dataset = MusicaDataset(df)

In [ ]:
def collate_fn(batch):
    """
    Input all text samples
    """

    texts = [r[0].replace('\n', ' ') for r in batch]

    batch_tokens = tokenizer.batch_encode_plus(texts,
                                    padding='max_length',
                                    truncation=True,
                                    max_length=256,
                                    return_tensors="pt")

    return (
        batch_tokens["input_ids"],
)

In [ ]:
dataloader = DataLoader(
                            dataset,
                            batch_size=256,
                            num_workers=mp.cpu_count(),
                            shuffle=True,
                            collate_fn=collate_fn
                          )

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained(model_ckpt)
model = BertModel.from_pretrained(model_ckpt)

Downloading:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/205k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/647 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
def generate_embeddings(model, dataloader):
    model.eval()
    model.to(device)
    embeddings_bert = None
    for batch in tqdm(dataloader):
        inputs = batch[0].to(device)
        with torch.no_grad():
            out_data = model(inputs)
            if embeddings_bert is None:
                embeddings_bert = out_data['pooler_output'].cpu().numpy()
            else:
                embeddings_bert = np.concatenate((embeddings_bert, out_data['pooler_output'].cpu().numpy()), axis=0)
    return embeddings_bert

In [ ]:
embeddings = generate_embeddings(model, dataloader)
folder = '/path/to/save/'
np.save(folder+'embedding.npy', embeddings)

100%|██████████| 54/54 [03:30<00:00,  3.89s/it]


#### Embedding Analyzes


What are the conclusions? (1-2 paragraphs)




### 2. Use a clustering algorithm to identify the possible groups present in the data set and to choose the size of k use the Elbow curve method. Also analyze the influence of the distance used in the chosen algorithm. Plot the generated groups using the information learned by the model.

In [ ]:
# TODO: Clustering. You can use scikit-learn
# You can modify the Class Clustering
class Clustering():
    #Define parameters
    def __init__(self):
        #replace for Clustering
        self.cluster = None

    def fit(self, X):
        pass

    def predict(self, X):
        pass

    def save_model(self, filename):
        """
        Save the model to disk
        """
        pickle.dump(self.cluster, open(filename, 'wb'))

    def load_model(self, filename):
        self.cluster = pickle.load(open(filename, 'rb'))

    def score(self, X):
        pass
        #choose you metric from sklearn and adds here


What are the conclusions? (1-2 paragraphs)




### 3. Use the test data to make a qualitative assessment of the model by comparing whether new data are clustered that have a certain similarity. Analyze the models generated through the Silhouete calculation and discuss the generated results.


What are the conclusions? (1-2 paragraphs)




## Submission

On Google Classroom, submit your Jupyter Notebook (in Portuguese or English).

**This activity is NOT individual, it must be done in pairs (two-person group).**